In [1]:
import sys
print("Python Version:", sys.version)

Python Version: 3.7.4 (default, Aug 13 2019, 20:35:49) 
[GCC 7.3.0]


In [2]:
#######################
# standard code block #
#######################

%config InlineBackend.figure_format = 'svg'

# auto reload imports that change
%load_ext autoreload
# only set to auto reload for marked imports
%autoreload 1

## Pull Data

### Connect to Database

In [3]:
from pymongo import MongoClient
%aimport credentials.cred
from credentials import cred


config = {
  'host': cred.mongo_host,
  'username': cred.mongo_user,
  'password': cred.mongo_pass,
  'authSource': cred.mongo_auth_db
}

# get a mongo client
client = MongoClient(**config)

# use the raw database
jre_raw = client.jre_raw
podcasts_raw = jre_raw.podcasts

### Get Podcasts

In [49]:
# possible keys = _id, name, url, number, text, source, desc
# query
ep_mongo = list(
    podcasts_raw
    .find({},{'_id':0})
)

In [50]:
names_list = [ep['name'] for ep in ep_mongo]
numbers_list = [ep['number'] for ep in ep_mongo]

## Clean

### Clean Names

In [27]:
import re

clean_names = [re.sub(r'^.*?\d+(\s*-\s*|\s*with\s*)','',name) for name in names_list]

In [57]:
# clean up the weird names
renames = {
    'JRE MMA Show #2':'Gastelum/Bisping fight', 'Podcast from a Car #2':'Cameron Hanes',
    'Podcast from a Car':'Bryan Callen','#319 â\x80\x93 Alex Honnold, Brian Redban':'Alex Honnold, Brian Redban',
    '#194 â\x80\x93 Jason Silva, Brian Redban':'Jason Silva, Brian Redban','PODCAST #12':'Joe Rogan', 
    ' Swapcast - Podcast On A Plane with John Dudley':'Swapcast - Podcast On A Plane with John Dudley'    
}

for i, name in enumerate(clean_names):
    if name in renames.keys():
        clean_names[i] = renames[name]
        print(f"renamed '{name}' to '{renames[name]}'")

renamed 'JRE MMA Show #2' to 'Gastelum/Bisping fight'
renamed 'Podcast from a Car #2' to 'Cameron Hanes'
renamed 'Podcast from a Car' to 'Bryan Callen'
renamed '#319 â Alex Honnold, Brian Redban' to 'Alex Honnold, Brian Redban'
renamed '#194 â Jason Silva, Brian Redban' to 'Jason Silva, Brian Redban'
renamed 'PODCAST #12' to 'Joe Rogan'
renamed ' Swapcast - Podcast On A Plane with John Dudley' to 'Swapcast - Podcast On A Plane with John Dudley'


### Put cleaned names into ep_mongo

In [64]:
for i,ep in enumerate(ep_mongo):
    ep['name'] = clean_names[i]

### Clean Numbers

In [75]:
# find nones
none_idx = [i for i,x in enumerate(numbers_list) if x is None]
none_idx

[381, 401, 405, 415, 424, 577, 864]

## Push up to cleaned podcast collection

In [78]:
# get the clean database
jre_clean = client.jre_clean
podcasts_clean = jre_clean.podcasts

In [79]:
# push the cleaned versions
podcasts_clean.insert_many(ep_mongo)

In [80]:
len(ep_mongo) == podcasts_clean.count_documents({})

True

In [81]:
client.close()